# Groupby

One of the most useful things about Pandas is the ability to group different datasets by values of a column and then perform an operation on the entire piece. This might not be very clear just yet but it should make a lot of sense soon. We are going to perform the same kind of analysis that we did in 7.1 for the Dow 30 Stocks. Begin by executing the code in the cell below. 

In [ ]:
# provided code
import pandas as pd
import pandas_datareader.data as web
import datetime as dt

# start date
start = dt.datetime(2017, 1, 1)
# end date
end = dt.datetime(2018, 12, 31)

# dow 30 components
# https://www.cnbc.com/dow-30/
dow_stocks = ['AXP', 'AAPL', 'BA', 'CAT', 
              'CSCO', 'CVX', 'DWDP',
              'XOM', 'GS', 'HD', 'IBM',
              'INTC', 'JNJ', 'KO', 'JPM',
              'MCD', 'MMM', 'MRK', 'MSFT',
              'NKE', 'PFE', 'PG', 'TRV',
              'UNH', 'UTX', 'VZ', 'V',
              'WBA', 'WMT', 'DIS']

# list to store dataframes
df_list = list()
for ticker in dow_stocks:
    ticker_df = web.DataReader(ticker, 'iex', start, end).reset_index()
    ticker_df['ticker'] = ticker
    df_list.append(ticker_df)
    
# create a final dataframe
complete_df = pd.concat(df_list)
complete_df.head()

Now we have one DataFrame which has the columns; ```date```, ```open```, ```high```, ```low```, ```close```, ```volume```, and ```ticker```. Frequently data will be disorganized like this which makes it difficult to work with. This is one way of resolving this kind of an issue. What we need to obtain is, for each ticker, a DataFrame of closing prices, simple moving averages, and positions for each day. Begin by typing the following in the cell below;
```python
df_gb = complete_df.groupby('ticker')
```

In [ ]:
# your code goes here

This will create a Pandas Groupby Object. This isn't something you can work with and access directly but to illustrate how useful it can be complete the following exercise;
<br><br>
#### Exercise
Write a For Loop where you iterate through a list of Tuples, the first element will be called ```ticker``` and the second will be ```temp_df```. The iterable item will be the ```df_gb``` object created above. At each iteration print out the ticker and the top of the DataFrame ```temp_df``` using the ```.head()``` method. Then print out 4 empty spaces to create spacing between each print.

In [ ]:
# your code goes here

As you can see from the above, at each iteration, we are working with a ```temp_df``` which only has data for the ```ticker``` we are working with.
<br><br>
#### Exercise
Now we are going to write a function which returns the daily positions (```True``` or ```False```) for each ticker. This will be called on the ```DataFrame``` using the ```.apply()``` method. This method should be called ```create_positions``` and takes 1 argument, ```ticker_data_df```. This will involve you returning a Pandas Series of the positions for the ticker, and you will have to use ```.set_index()``` to set the index of the returning Series to be ```'date'```. Setting the index should happen first before anything else in the function.

In [ ]:
# your code goes here

Now lets run the code below to obtain the results

In [ ]:
# provided code
positions = df_gb.apply(create_positions)
positions.head()

## Groupby Particulars
Our ```positions``` Series is now a Multi-Indexed Series consisting of both each ```ticker``` and each ```date```. When working with a Multi-Index we give each index component a ```level``` which denotes the heirarchy of the data. In our example our ```ticker``` is at ```level``` number ```0``` and ```date``` is at ```level``` number ```1```. We will now learn how to select from a Multi-Indexed DataFrame or Series.
<br><br>
### Getting Index Level Values
The first thing we are going to look at is how to obtain the values of these index levels. This is done through the ```get_level_values``` method which can be called. Try executing the code below;

In [ ]:
# provided code
tickers = list(set(positions.index.get_level_values(0)))
tickers

The code above has now provided us with the 30 original tickers we used in the beginning. The same thing works for getting the dates.
<br><br>
### Selecting by Index Values
Now if we needed to select a single ```ticker``` we could do so with our standard methodology of ```.loc[rows]```. Try the code below to obtain the series for both ```AAPL``` and ```BA```.

In [ ]:
# provided code
positions.loc[['AAPL', 'BA']]

If we needed to select only certain dates for each of these securities we could so using a Tuple containing two Lists, the first List would be Tickers and the second would be Dates. See the example below;

In [ ]:
# provided code

positions.loc[(['AAPL', 'BA'], ['2017-01-03', '2017-02-01'])]

Note that because our dates are in a String type we can use a String to access them. If they were of a Datetime type we would need to use Datetime objects. Notice that our Tuple ```(['AAPL', 'BA'], ['2017-01-03', '2017-02-01'])``` puts the Tickers first (since they are ```level 0```) and then the dates after.
<br><br>
### Selecting all of Level 0 but only some of Level 1
If we needed to narrow down our selection in one ```level``` but not another we could leverage the ```slice(None)``` operation in our selection. Wherever Pandas sees ```slice(None)``` it knows to select everything from that ```level```. Execute the code in the cell below to select all of the tickers but only for the dates ```['2017-01-03', '2017-02-01']```.

In [ ]:
# provided code
positions.loc[(slice(None), ['2017-01-03', '2017-02-01'])]

If we tried the following code we would return an empty series;
```python
positions.loc[['2017-01-03', '2017-02-01']]
```
This is because Pandas things you are looking for the dates ```['2017-01-03', '2017-02-01']``` in ```level 0``` which they don't exist in.